In [1]:
import pandas as pd

In [2]:
reports_dir_tmpl = "reports_seed_{}"
report_dir_tmpl = "delay_{}_drop_{}"

In [3]:
import os
from pathlib import Path

In [4]:
cwd = os.getcwd()

In [9]:
# Define the path to the reports directory
reports_dir = Path(cwd).parent / reports_dir_tmpl.format(42)
reports_dir

PosixPath('/Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42')

In [10]:
# Define lists for delay and drop_rate values
delays = ["5ms", "50ms", "100ms", "200ms", "500ms"]
drop_rates = [0.01, 0.05, 0.1, 0.2, 0.3]

In [11]:
report_dir = reports_dir / report_dir_tmpl.format(delays[3], drop_rates[1])
report_dir

PosixPath('/Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42/delay_200ms_drop_0.05')

In [12]:
if report_dir.is_dir():
    print(report_dir)
    stderr_path = report_dir / "stderr.log"
    filtered_stderr_path = report_dir / "filtered_stderr.log"

    if stderr_path.exists():
        print(stderr_path)
        # Read the stderr.log file
        with open(stderr_path, "r", encoding="utf-8") as file:
            lines = file.read().split("\n")

        # Keep only lines that start with "event:"
        filtered_lines = [line for line in lines if line.startswith("event:")]

        # Write the filtered lines to a new file
        with open(filtered_stderr_path, "w", encoding="utf-8") as file:
            file.writelines(filtered_lines)

        print(f"Filtered {stderr_path} and saved to {filtered_stderr_path}")

/Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42/delay_200ms_drop_0.05
/Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42/delay_200ms_drop_0.05/stderr.log
Filtered /Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42/delay_200ms_drop_0.05/stderr.log and saved to /Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42/delay_200ms_drop_0.05/filtered_stderr.log


In [13]:
filtered_lines[0]

'event:on_packet_sent,time_sent:0:00:00.200000,under_utilized:true,bytes_in_flight:191,congestion_window:12000'

In [14]:
def get_all_lines_from_report_dir(report_dir) -> list:
    filtered_lines = []
    if report_dir.is_dir():
        stderr_path = report_dir / "stderr.log"
        print(f"Reading {stderr_path}...")
        if stderr_path.exists():
            # Read the stderr.log file
            with open(stderr_path, "r", encoding="utf-8") as file:
                lines = file.read().split("\n")
            # Keep only lines that start with "event:"
            filtered_lines = [line for line in lines if line.startswith("event:")]
    return filtered_lines

In [15]:
filtered_lines = get_all_lines_from_report_dir(report_dir)

Reading /Users/luzhaoyan/workspace/github/s2n-quic/quic/s2n-quic-sim/reports_seed_42/delay_200ms_drop_0.05/stderr.log...


In [16]:
len(filtered_lines)

495867

In [17]:
def filter_out(lines: list, event: str) -> list:
    filtered_lines = [line for line in lines if event not in line]
    return filtered_lines

In [18]:
filtered_lines = filter_out(filtered_lines, "on_rtt_update")

In [19]:
len(filtered_lines)

336358

In [20]:
on_packet_sent_line = ""
for line in filtered_lines:
    if "on_packet_sent" in line:
        on_packet_sent_line = line
        break
on_packet_sent_line

'event:on_packet_sent,time_sent:0:00:00.200000,under_utilized:true,bytes_in_flight:191,congestion_window:12000'

In [21]:
on_packet_lost_line = ""
for line in filtered_lines:
    if "on_packet_lost" in line:
        on_packet_lost_line = line
        break
on_packet_lost_line

'event:on_packet_lost,timestamp:0:00:02.924665,under_utilized:true,lost_bytes:53,persistent_congestion:false,new_loss_burst:true,bytes_in_flight:1601,congestion_window:9828'

In [22]:
on_ack_line = ""
for line in filtered_lines:
    if "on_ack" in line:
        on_ack_line = line
        break
on_ack_line

'event:on_ack,newest_acked_time_sent:0:00:00.200000,under_utilized:true,bytes_acknowledged:191,ack_receive_time:0:00:00.600000,bytes_in_flight:1009,congestion_window:12000'

In [23]:
columns = [
    "event",
    "timestamp",
    "lost_bytes",
    "bytes_acknowledged",
    "bytes_in_filght",
    "congestion_window",
]

In [24]:
def format_line(line):
    if "on_packet_sent" in line:
        # event:on_packet_sent,time_sent:0:00:00.200000,under_utilized:true,bytes_in_flight:191,congestion_window:12000
        cols = line.split(",")
        record = {
            columns[0]: cols[0].split(":")[1],
            columns[1]: cols[1].split(":", 1)[1],  # time_sent
            columns[2]: 0,
            columns[3]: 0,
            columns[4]: cols[3].split(":")[1],
            columns[5]: cols[4].split(":")[1],
        }
    elif "on_packet_lost" in line:
        # event:on_packet_lost,timestamp:0:00:02.924665,under_utilized:true,lost_bytes:53,persistent_congestion:false,new_loss_burst:true,bytes_in_flight:1601,congestion_window:9828
        cols = line.split(",")
        record = {
            columns[0]: cols[0].split(":")[1],
            columns[1]: cols[1].split(":", 1)[1],  # timestamp
            columns[2]: cols[3].split(":")[1],  # lost_bytes
            columns[3]: 0,
            columns[4]: cols[6].split(":")[1],
            columns[5]: cols[7].split(":")[1],
        }
    elif "on_ack" in line:
        # event:on_ack,newest_acked_time_sent:0:00:00.200000,under_utilized:true,bytes_acknowledged:191,ack_receive_time:0:00:00.600000,bytes_in_flight:1009,congestion_window:12000
        cols = line.split(",")
        record = {
            columns[0]: cols[0].split(":")[1],
            columns[1]: cols[4].split(":", 1)[1],  # ack_receive_time
            columns[2]: 0,
            columns[3]: cols[3].split(":")[1],  # bytes_acknowledged
            columns[4]: cols[5].split(":")[1],
            columns[5]: cols[6].split(":")[1],
        }
    else:
        raise Exception(f"Unsupported line: {line}")
    return record

In [30]:
records = list(map(format_line, filtered_lines))

In [31]:
records[4]

{'event': 'on_packet_sent',
 'timestamp': '0:00:00.600000',
 'lost_bytes': 0,
 'bytes_acknowledged': 0,
 'bytes_in_filght': '1472',
 'congestion_window': '12000'}

In [32]:
df = pd.DataFrame.from_records(records)

In [33]:
df

,event,timestamp,lost_bytes,bytes_acknowledged,bytes_in_filght,congestion_window
0,on_packet_sent,0:00:00.200000,0,0,191,12000
1,on_packet_sent,0:00:00.200000,0,0,1200,12000
2,on_ack,0:00:00.600000,0,191,1009,12000
3,on_ack,0:00:00.600000,0,1009,0,12000
4,on_packet_sent,0:00:00.600000,0,0,1472,12000
...,...,...,...,...,...,...
336353,on_ack,7:47:19.944427,0,53,106,10066
336354,on_ack,7:47:20.147135,0,53,53,10066
336355,on_packet_sent,7:47:20.295131,0,0,106,10066
336356,on_ack,7:47:20.354443,0,53,53,10066


In [40]:
def ts2ms(timestamp: str) -> int:
    h, m, s = timestamp.split(":")
    m = int(m) + int(h) * 60
    ms = round(float(s) * 1000) + m * 60 * 1000
    return ms

In [42]:
df["timestamp"] = df["timestamp"].apply(ts2ms)

In [43]:
df["congestion_window"].unique()

array(['12000', '13360', '14040', '9828', '10066'], dtype=object)

In [44]:
df[df["event"] == "on_packet_lost"]

,event,timestamp,lost_bytes,bytes_acknowledged,bytes_in_filght,congestion_window
36,on_packet_lost,2925,53,0,1601,9828
174,on_packet_lost,6888,59,0,472,10066
208,on_packet_lost,7794,61,0,491,10066
212,on_packet_lost,7904,61,0,487,10066
261,on_packet_lost,9094,60,0,416,10066
...,...,...,...,...,...,...
336153,on_packet_lost,28021652,53,0,106,10066
336183,on_packet_lost,28024483,53,0,106,10066
336193,on_packet_lost,28025461,53,0,106,10066
336200,on_packet_lost,28026269,54,0,53,10066


In [45]:
df[35:38]

,event,timestamp,lost_bytes,bytes_acknowledged,bytes_in_filght,congestion_window
35,on_packet_sent,2865,0,0,1654,14040
36,on_packet_lost,2925,53,0,1601,9828
37,on_packet_sent,2946,0,0,1660,9828


In [49]:
df_one_hot = pd.get_dummies(df, columns=["event"], dtype=int)

# Specify the column to move
col_to_move = "congestion_window"

# Move the column to the end
df_one_hot = df_one_hot[
    [col for col in df_one_hot if col != col_to_move] + [col_to_move]
]

In [50]:
df_one_hot

,timestamp,lost_bytes,bytes_acknowledged,bytes_in_filght,event_on_ack,event_on_packet_lost,event_on_packet_sent,congestion_window
0,200,0,0,191,0,0,1,12000
1,200,0,0,1200,0,0,1,12000
2,600,0,191,1009,1,0,0,12000
3,600,0,1009,0,1,0,0,12000
4,600,0,0,1472,0,0,1,12000
...,...,...,...,...,...,...,...,...
336353,28039944,0,53,106,1,0,0,10066
336354,28040147,0,53,53,1,0,0,10066
336355,28040295,0,0,106,0,0,1,10066
336356,28040354,0,53,53,1,0,0,10066


In [48]:
df_one_hot.to_csv("test.csv")